<a href="https://colab.research.google.com/github/thiagomantuani/meli-data-challenge-2021/blob/main/2_model_meli_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tweedie
!pip install scikit-learn --upgrade
!pip install tqdm

# **Imports**

In [ ]:
from google.colab                    import drive   
import pandas                        as pd
import numpy                         as np
import seaborn                       as sns
import datetime
from sklearn import linear_model
import scipy.stats as st
import pdb
import os
import gc
import tweedie
import tqdm
from scipy.stats import norm
from sklearn.preprocessing import PowerTransformer

%matplotlib inline

In [ ]:
drive.mount('/gdrive')
path = '/gdrive/My Drive/DataSets/meli2021/'
os.chdir(path)

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


# **Functions**


In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                       df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage(deep=True).sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
def pred_list_to_distro(pred_list, wei=False, total_days=30, phi=2, power=1.5):
    distros = dict()
    for mu in tqdm.tqdm(range(1,total_days+1),total=total_days):
        if wei:
            distros[mu] = [st.norm.cdf(days, loc=mu, scale=1) for days in range(0,total_days+1,1)]
            #distros[mu] = [st.norm.cdf(days+1, loc=mu,scale=1)-st.norm.cdf(days,loc=mu,scale=1) for days in range(0,total_days+1,1)]
        else:
            distros[mu] = [tweedie.tweedie(p=power, mu=mu, phi=phi).cdf(days) for days in range(0,total_days+1,1)]

        if np.sum(distros[mu]) == 0:
            distros[mu] = 1/total_days
        else:          
          if np.sum(distros[mu]) > 0:           
              distros[mu] = np.diff(distros[mu])
              distros[mu] = np.round(distros[mu] / np.sum(distros[mu]), 4)
          else:
              distros[mu] = distros[mu][1:]
              distros[mu][-1] = 1
        
    
    prob_array = np.zeros((pred_list.shape[0], total_days))

    for row, mu in enumerate(pred_list):
        prob_array[row, :] = distros[mu]

    return prob_array

In [ ]:
def calc_probs(pred_list, std_stock=8.652240203417234, total_days=30):
  distros = dict()
  for j in tqdm.tqdm(pred_list,total=len(pred_list)):
    norm_dist= norm(j,std_stock)
    probs = []
    for i in range(1,total_days+1):
      probs.append(norm_dist.cdf(i+1)-norm_dist.cdf(i))
    if np.sum(probs)==0:
      probs = list(np.ones(total_days)/30)
    distros[j] = np.asarray(probs)

  prob_array = np.zeros((len(pred_list),total_days))
  for row, mu in enumerate(pred_list):
    prob_array[row,:] = distros[mu]

  return prob_array

In [ ]:
def preprocessing(df):
  df = df.reset_index()
  categorys = ['item_domain_id', 'currency', 'listing_type', 'shipping_logistic_type', 'shipping_payment', 'site_id']
  for cat in categorys:
    df[cat] = df[cat].astype('category').cat.codes
  df.loc[df['minutes_active']==0,'active'] = 0
  df['active'] = df['active'].fillna(1).copy()  
  df['date'] = pd.to_datetime(df.date)
  df = reduce_mem_usage(df)
  
  return df

# **Load data**

In [ ]:
df = pd.read_csv(path+'/train_data_full.csv.gz',compression="gzip",index_col=0)
test = pd.read_csv(path+'/test_data.csv',index_col=0).squeeze()

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
df.head(3)

,date,sold_quantity,current_price,currency,listing_type,shipping_logistic_type,shipping_payment,minutes_active,item_domain_id,site_id
sku,,,,,,,,,,
464801,2021-02-01,0,156.78,REA,classic,fulfillment,free_shipping,1440.0,MLB-NEBULIZERS,MLB
464801,2021-02-02,0,156.78,REA,classic,fulfillment,free_shipping,1440.0,MLB-NEBULIZERS,MLB
464801,2021-02-03,0,156.78,REA,classic,fulfillment,free_shipping,1440.0,MLB-NEBULIZERS,MLB


# **Data description**

## **Data dimension**

In [ ]:
df.shape

(37660279, 10)

## **Data type**

In [ ]:
df.dtypes

date                       object
sold_quantity               int64
current_price             float64
currency                   object
listing_type               object
shipping_logistic_type     object
shipping_payment           object
minutes_active            float64
item_domain_id             object
site_id                    object
dtype: object

## **Checking NaN**


In [ ]:
df.isnull().sum()

date                        0
sold_quantity               0
current_price               0
currency                    0
listing_type                0
shipping_logistic_type      0
shipping_payment            0
minutes_active              0
item_domain_id            177
site_id                     0
dtype: int64

In [ ]:
df['item_domain_id'].value_counts()

MLB-SUPPLEMENTS               786063
MLM-CELLPHONE_COVERS          592007
MLB-HAIR_TREATMENTS           266634
MLM-T_SHIRTS                  254175
MLM-HEADPHONES                247016
                               ...  
MLM-GAME_CONSOLE_MULTITAPS         7
MLA-GLOVES_AND_MITTENS             6
MLA-MINI_BAG_SEALERS               6
MLA-PET_FOOTWEAR                   5
MLA-RADIO_BASE_STATIONS            1
Name: item_domain_id, Length: 8408, dtype: int64

In [ ]:
nulls = df[df['item_domain_id'].isna()]

In [ ]:
nulls.reset_index()['sku'].value_counts()

454273    59
459892    59
553503    59
Name: sku, dtype: int64

In [ ]:
df['item_domain_id'].fillna(df['item_domain_id'].mode()[0],inplace=True)

In [ ]:
df.dtypes

date                       object
sold_quantity               int64
current_price             float64
currency                   object
listing_type               object
shipping_logistic_type     object
shipping_payment           object
minutes_active            float64
item_domain_id             object
site_id                    object
dtype: object

In [ ]:
df['date'] = pd.to_datetime(df['date'])
num_attributes = df.select_dtypes(include = ['int32', 'int64', 'float64'])

cat_attributes = df.select_dtypes(exclude = ['int32', 'int64', 'float64', 'datetime64[ns]'])

In [ ]:
ct1 = pd.DataFrame( num_attributes.apply(np.mean)).T
ct2 = pd.DataFrame( num_attributes.apply(np.median)).T

d1 = pd.DataFrame( num_attributes.apply(np.std)).T
d2 = pd.DataFrame( num_attributes.apply(min)).T
d3 = pd.DataFrame( num_attributes.apply(max)).T
d4 = pd.DataFrame( num_attributes.apply(lambda x: x.max() - x.min() ) ).T
d5 = pd.DataFrame( num_attributes.apply(lambda x: x.skew() ) ).T
d6 = pd.DataFrame( num_attributes.apply(lambda x: x.kurtosis() ) ).T

m = pd.concat([d2, d3, d4, ct1, ct2, d1, d5, d6]).T.reset_index()
m.columns = ['atributos', 'min', 'max', 'intervalo', 'media', 'mediana', 'desvio', 'skew', 'curtose']

m

,atributos,min,max,intervalo,media,mediana,desvio,skew,curtose
0,sold_quantity,0.00,6.951000e+03,6.951000e+03,0.990093,0.0,9.989535e+00,178.549615,62641.787827
1,current_price,0.87,1.000000e+09,1.000000e+09,2357.780463,119.0,1.263914e+06,789.185694,624305.281067
2,minutes_active,0.00,1.440000e+03,1.440000e+03,1043.414678,1440.0,6.302455e+02,-1.008095,-0.940990


In [ ]:
cat_attributes.apply(lambda x: x.unique().shape[0])

currency                     4
listing_type                 2
shipping_logistic_type       3
shipping_payment             2
item_domain_id            8408
site_id                      3
dtype: int64

In [ ]:
del num_attributes
del cat_attributes
del nulls
gc.collect()

116

# **Model**

In [ ]:
df = preprocessing(df)

Mem. usage decreased to 1041.55 Mb (47.3% reduction)


In [ ]:
cols_remove_for_train = ['date','sold_quantity']

In [ ]:
%%time
mdl = linear_model.TweedieRegressor(power=0.0,verbose=1,max_iter=400)
pt = PowerTransformer()
mdl.fit(df.drop(cols_remove_for_train,axis=1), pt.fit_transform( df[['sold_quantity']] ))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


CPU times: user 3min 37s, sys: 20.1 s, total: 3min 57s
Wall time: 2min 59s


In [ ]:
test_df = df[df['date']=='2021-03-31']
test_df = test_df[test_df['sku'].isin(test.index)]

In [ ]:
y_pred =  mdl.predict(test_df.drop(cols_remove_for_train,axis=1)) 
y_pred2 = pt.inverse_transform(y_pred.reshape(-1,1))

In [ ]:
tmp = test_df[['sku']].copy()
tmp['predict'] = y_pred2
tmp['real_stock'] = tmp['sku'].map(test)
tmp['days_to_stockout'] = (tmp['real_stock']/tmp['predict']).fillna(30.).clip(1,30).astype(int)

# **Submission**

In [ ]:
prob = pred_list_to_distro(tmp['days_to_stockout'].values)

100%|██████████| 30/30 [00:01<00:00, 15.88it/s]


In [ ]:
pd.DataFrame(prob).round(4).to_csv('sub41.csv.gz',header=False,index=False,compression='gzip')

In [ ]:
sub = pd.read_csv('sub41.csv.gz',header=None)
sub

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,0.0180,0.0221,0.0259,0.0293,0.0324,0.0351,0.0373,0.0392,0.0406,0.0416,0.0423,0.0426,0.0425,0.0422,0.0417,0.0409,0.0399,0.0387,0.0374,0.0360,0.0345,0.0330,0.0314,0.0298,0.0282,0.0266,0.0250,0.0235,0.0219,0.0205
1,0.0092,0.0118,0.0144,0.0171,0.0197,0.0223,0.0248,0.0272,0.0294,0.0315,0.0334,0.0351,0.0367,0.0380,0.0392,0.0402,0.0410,0.0415,0.0420,0.0422,0.0423,0.0422,0.0419,0.0416,0.0411,0.0405,0.0398,0.0390,0.0381,0.0371
2,0.0087,0.0112,0.0138,0.0164,0.0189,0.0215,0.0240,0.0263,0.0286,0.0307,0.0327,0.0345,0.0361,0.0376,0.0389,0.0400,0.0409,0.0416,0.0421,0.0425,0.0427,0.0427,0.0426,0.0424,0.0420,0.0415,0.0409,0.0402,0.0394,0.0386
3,0.0087,0.0112,0.0138,0.0164,0.0189,0.0215,0.0240,0.0263,0.0286,0.0307,0.0327,0.0345,0.0361,0.0376,0.0389,0.0400,0.0409,0.0416,0.0421,0.0425,0.0427,0.0427,0.0426,0.0424,0.0420,0.0415,0.0409,0.0402,0.0394,0.0386
4,0.0087,0.0112,0.0138,0.0164,0.0189,0.0215,0.0240,0.0263,0.0286,0.0307,0.0327,0.0345,0.0361,0.0376,0.0389,0.0400,0.0409,0.0416,0.0421,0.0425,0.0427,0.0427,0.0426,0.0424,0.0420,0.0415,0.0409,0.0402,0.0394,0.0386
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
551467,0.0087,0.0112,0.0138,0.0164,0.0189,0.0215,0.0240,0.0263,0.0286,0.0307,0.0327,0.0345,0.0361,0.0376,0.0389,0.0400,0.0409,0.0416,0.0421,0.0425,0.0427,0.0427,0.0426,0.0424,0.0420,0.0415,0.0409,0.0402,0.0394,0.0386
551468,0.0087,0.0112,0.0138,0.0164,0.0189,0.0215,0.0240,0.0263,0.0286,0.0307,0.0327,0.0345,0.0361,0.0376,0.0389,0.0400,0.0409,0.0416,0.0421,0.0425,0.0427,0.0427,0.0426,0.0424,0.0420,0.0415,0.0409,0.0402,0.0394,0.0386
551469,0.0087,0.0112,0.0138,0.0164,0.0189,0.0215,0.0240,0.0263,0.0286,0.0307,0.0327,0.0345,0.0361,0.0376,0.0389,0.0400,0.0409,0.0416,0.0421,0.0425,0.0427,0.0427,0.0426,0.0424,0.0420,0.0415,0.0409,0.0402,0.0394,0.0386
551470,0.0196,0.0239,0.0279,0.0314,0.0345,0.0371,0.0392,0.0409,0.0421,0.0429,0.0433,0.0433,0.0430,0.0425,0.0416,0.0406,0.0393,0.0380,0.0365,0.0349,0.0332,0.0315,0.0298,0.0281,0.0264,0.0248,0.0232,0.0216,0.0201,0.0186


# **References**

https://www.kaggle.com/rinnqd/reduce-memory-usage

https://github.com/ledmaster/english_tutorials

https://ml-challenge.mercadolibre.com/recipe/2021/baseline